In [38]:
suppressMessages(library(rwwa))
png_res <- 240

# Functions

## Automate synthesis

In [39]:
full_synthesis <- function(res_obs, res_mdl, dI_type = "abs", cf_nm = "pi", uw = F) {
    
    # when adding to rwwa package, make column names an optional argument but use these defaults if missing
    # also should add the unweighted values to the tables

    # Synthesise change in intensity
    obs_dI <- res_obs[,grepl(paste0(dI_type, ".", cf_nm, "_"), colnames(res_obs))]
    mdl_dI <- res_mdl[,grepl(paste0("attr_dI.",dI_type,".",cf_nm, "_"), colnames(res_mdl))]

    attr_dI <- synthesis(obs_dI, mdl_dI, synth_type = dI_type)
    proj_dI <- synthesis(obs_in = NA, res_mdl[,grepl(paste0("proj_dI.",dI_type), colnames(res_mdl))], synth_type = dI_type)$df

    # Synthesise PR
    obs_pr <- infer_infinite(res_obs[,grepl(paste0("PR.", cf_nm), colnames(res_obs))])
    mdl_pr <- infer_infinite(res_mdl[,grepl("attr_PR", colnames(res_mdl))])
    
    attr_pr <- synthesis(obs_pr, mdl_pr, synth_type = "PR")
    proj_pr <- synthesis(obs_in = NA, infer_infinite(res_mdl[,grepl("proj_PR", colnames(res_mdl))]), synth_type = "PR")$df

    if (uw) {
        attr_dI <- rbind(attr_dI$df, setNames(c("synth_uw", "Unweighted", attr_dI$uw, attr_dI$df[nrow(attr_dI$df), 6:7], rep(NA, 2)), colnames(attr_dI$df)))
        attr_pr <- rbind(attr_pr$df, setNames(c("synth_uw", "Unweighted", attr_pr$uw, attr_pr$df[nrow(attr_pr$df), 6:7], rep(NA, 2)), colnames(attr_pr$df)))
    } else {
        attr_dI <- attr_dI$df
        attr_pr <- attr_pr$df
    }

    # compile the results of the attribution analysis
    tbl_attr <- t(sapply(attr_dI$group[grepl("synth", attr_dI$group)], function(cnm) {
        di = signif(attr_dI[attr_dI$group == cnm,c("est", "lower", "upper")], 3)
        pr = signif(attr_pr[attr_pr$group == cnm,c("est", "lower", "upper")], 3)
        
        sapply(list("di" = di, "pr" = pr), function(r) paste0(r[1], " (",r[2], ", ", r[3], ")"))
    }))

    # compile projected results
    tbl_proj <- t(sapply(c("model_synth"), function(cnm) {
        di = signif(proj_dI[proj_dI$group == cnm,c("est", "lower", "upper")], 3)
        pr = signif(proj_pr[proj_pr$group == cnm,c("est", "lower", "upper")], 3)
        
        sapply(list("di" = di, "pr" = pr), function(r) paste0(r[1], " (",r[2], ", ", r[3], ")"))
    }))
    rownames(tbl_proj) <- c("future")

    # return single table
    return(list("dI_attr" = attr_dI, "dI_proj" = proj_dI, "PR_attr" = attr_pr, "PR_proj" = proj_pr,
                "table" = rbind(tbl_attr, tbl_proj)[,c("pr", "di")]))
}

## Clean up plotting

Not necessary in this study, since the bars don't end close to zero

In [14]:
# # modified plotting function with butt ends, instead of square
# plot_synthesis <- function (synth, xlim, lwd = 10, xlab = "", main = "", add_space = T, 
#     log = NA, hide_labels = F) {
#     gcols = c(obs = adjustcolor("blue", 0.5), obs_synth = "blue", 
#         models = adjustcolor("red", 0.5), model_synth = "red", 
#         synth = "magenta")
#     if (is.na(log)) {
#         if (!is.null(synth$synth)) {
#             if (synth$synth_type == "PR") {
#                 logaxs <- "x"
#             }
#             else {
#                 logaxs <- ""
#             }
#         }
#         else {
#             logaxs <- ""
#         }
#     }
#     else {
#         if (log) {
#             logaxs <- "x"
#         }
#         else {
#             logaxs <- ""
#         }
#     }
#     if (is(synth, "list")) 
#         synth <- synth$df
#     if (missing(xlim)) {
#         if (logaxs == "x") {
#             xlim <- exp(range(pretty(log(as.numeric(unlist(synth[, 
#                 c("lower", "upper", "l_wb", "u_wb")]))))))
#         }
#         else {
#             xlim <- range(pretty(as.numeric(unlist(synth[, c("lower", 
#                 "upper", "l_wb", "u_wb")]))))
#         }
#     }
#     if (is.numeric(synth$group)) 
#         synth$group <- names(gcols)[synth$group]
#     nobs <- sum(synth$group == "obs")
#     nmod <- sum(synth$group == "models")
#     if (add_space & nobs > 0) {
#         yy <- c(rev(0:nobs + nmod + 4), rev(0:nmod + 2), 0)
#     }
#     else {
#         yy <- nrow(synth):1
#     }
#     if (logaxs == "x") {
#         vline <- 1
#     }
#     else {
#         vline <- 0
#     }
#     plot(0, type = "n", xlim = xlim, ylim = range(yy) + c(-0.5, 
#         0.5), log = logaxs, yaxt = "n", ylab = "", xlab = xlab, 
#         main = main)
#     grid(ny = NA, col = adjustcolor("black", 0.1), lty = 1)
#     abline(v = vline, lty = 1)
#     gcols <- gcols[synth$group]
#     segments(y0 = yy, x0 = synth$l_wb, x1 = synth$u_wb, lwd = lwd + 2, col = "black", lend = 1)
#     segments(y0 = yy, x0 = synth$l_wb, x1 = synth$u_wb, lwd = lwd - 2, col = "white", lend = 1)
#     segments(y0 = yy, x0 = synth$lower, x1 = synth$upper, lwd = lwd, 
#         col = gcols, lend = 1)
#     points(synth$est, yy, pch = 21, bg = gcols, lwd = 2, cex = lwd/10)
#     if (!hide_labels) 
#         axis(2, at = yy, labels = synth$model, las = 1)
# }

# Synthesis

In [44]:
varnm <- "rx1day-ondjfm"
rnm <- "s"
cf_nm = "pi"

In [47]:
# variables that don't need to be changed in this case
mtype <- "gmst+nao"
dI_type = "rel"

restype = list("rx1day-ondjfm_s" = "good", "rx1day-ondjfm_n" = "good", "rx90day_s" = "notbad", "rx90day_n" = "notbad")[[paste0(varnm,"_",rnm)]]

# load & clean all the data
res_obs <- read.csv(paste0("res-obs_",varnm,"_",rnm,".csv"), row.names = "X")
res_obs <- res_obs[sapply(strsplit(rownames(res_obs), "_"), "[", 2) == mtype,]
rownames(res_obs) <- sapply(strsplit(rownames(res_obs), "_"), "[", 1)
res_obs <- res_obs[c("eobs", "era5"),]

res_cordex <- read.csv(paste0("res-cordex_",varnm,"_",rnm,"_",mtype,"_with-eval.csv"), row.names = "X.1")
if (restype == "good") {
    res_cordex <- res_cordex[res_cordex$overall == "good",]
} else if (restype == "notbad") {
    res_cordex <- res_cordex[res_cordex$overall != "bad",]
}
res_mdl <- res_cordex

res_hr <- read.csv(paste0("res-highresmip_",varnm,"_",rnm,"_",mtype,"_with-eval.csv"), row.names = "X.1")
if(sum(res_hr$overall == "good") >5) {
    res_hr <- res_hr[res_hr$overall == "good",]
} else {
    res_hr <- res_hr[res_hr$overall != "bad",]
}
# manually filter out duplicated runs - keep 'good' rather than 'reasonable', then take higher-resolution version
if (rnm == "s" & varnm == "rx1day-ondjfm") {
    res_hr <- res_hr[c('CMCC-CM2-VHR4','EC-Earth3P','HadGEM3-GC31-MM','MPI-ESM1-2-XR'),]
} else if (rnm == "n" & varnm == "rx1day-ondjfm") {
    res_hr <- res_hr[c('CNRM-CM6-1-HR','EC-Earth3P','HadGEM3-GC31-HM','MPI-ESM1-2-XR'),]
}

res_mdl <- rbind.fill(res_cordex, res_hr)
rownames(res_mdl) <- c(rownames(res_cordex), rownames(res_hr))

In [48]:
# run synthesis
synth <- full_synthesis(res_obs, res_mdl, dI_type = dI_type, cf_nm = cf_nm, uw = T)

synth$table

# overwrite plotting limits
xlim_PR <- c(1e-3, 1e+3)

xlim_dI <- range(pretty(unlist(rbind(synth$dI_attr, synth$dI_proj)[,3:7])))

# # overwrite plotting limits
# xlim_PR <- list("rx1day-ondjfm_s" = c(0.1, 100),
#                 "rx1day-ondjfm_n" = c(0.1, 100))[[paste0(varnm, "_", rnm)]]

# xlim_dI <- list("rx1day-ondjfm_s" = c(-1, 3),
#                 "rx1day-ondjfm_n" = c(-5, 15))[[paste0(varnm, "_", rnm)]]

# save all tables
write.csv(synth$dI_attr, paste0("synth/synth-attr-dI_",varnm,"_",rnm,"_",cf_nm,"_",mtype,"_",restype,".csv"), row.names = F)
write.csv(synth$PR_attr, paste0("synth/synth-attr-PR_",varnm,"_",rnm,"_",cf_nm,"_",mtype,"_",restype,".csv"), row.names = F)

write.csv(synth$dI_proj, paste0("synth/synth-proj-dI_",varnm,"_",rnm,"_",cf_nm,"_",mtype,"_",restype,".csv"), row.names = F)
write.csv(synth$PR_proj, paste0("synth/synth-proj-PR_",varnm,"_",rnm,"_",cf_nm,"_",mtype,"_",restype,".csv"), row.names = F)

# table of results to paste into document
write.csv(synth$table, paste0("synth/synth-table_",varnm,"_",rnm,"_",cf_nm,"_",mtype,".csv"))

# figures for past change
png(paste0("fig/synth-attr_",varnm,"_",rnm,"_",cf_nm,"_",mtype,"_",restype,".png"), height = png_res * 3, width = png_res * 5); {
    prep_window(c(1,2), oma = c(0,20,0,0), res = 140, mar = c(3,1,2,1), h = 7, w = 5)
    plot_synthesis(synth$PR_attr[!synth$PR_attr$group == "synth_uw",], hide_labels = F, main = "(a) Probability ratio", xlim = xlim_PR, log = T)
    plot_synthesis(synth$dI_attr[!synth$PR_attr$group == "synth_uw",], hide_labels = T, main = "(b) Change in intensity", xlim = xlim_dI)
    
    # mtext(paste0(varnm, " - ", rnm), outer = T, side = 3, font = 2)
}; dev.off()

# figures for future change
png(paste0("fig/synth-proj_",varnm,"_",rnm,"_",cf_nm,"_",mtype,"_",restype,".png"), height = png_res * 3, width = png_res * 5); {
    prep_window(c(1,2), oma = c(0,20,0,0), res = 140, mar = c(3,1,2,1), h = 7, w = 5)
    plot_synthesis(synth$PR_proj, hide_labels = F, main = "(a) Probability ratio", xlim = xlim_PR, log = T)
    plot_synthesis(synth$dI_proj, hide_labels = T, main = "(b) Change in intensity", xlim = xlim_dI)
    
    # mtext(paste0(varnm, " - ", rnm), outer = T, side = 3, font = 2)
}; dev.off()

,pr,di
obs_synth,"23.9 (0.503, 54800)","35.5 (-2.94, 86.8)"
model_synth,"1.03 (0.337, 11)","1.19 (-15.1, 22)"
synth,"1.33 (0.285, 32)","8.36 (-13.3, 35.8)"
synth_uw,"4.95 (0.289, 1510)","17.1 (-10.3, 52.3)"
future,"1.12 (0.517, 2.38)","1.57 (-9.32, 12.8)"


agg_record_1792569771 
                    2

agg_record_1792569771 
                    2

## Quick synthesis of each model ensemble only for results table